In [ ]:
#|default_exp tree

# Game tree

> Fill in a module description here

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
from fastcore.all import *
import polvo as pv
import truco as tr
import numpy as np
import graphviz

/home/lgvaz/miniconda3/envs/truco/lib/python3.10/site-packages/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#|export
def get_bots(round_rank, hand11, hand12, hand21, hand22):
    round_card_pool = tr.round_card_pool(hand11+hand12+hand21+hand22, faceup)
    
    round_orders = list(itertools.product(itertools.permutations([0,1,2]), repeat=4))
    all_bots = []
    for round_order in round_orders:
        bots = [tr.FixedOrderBotPlayer(t, o) for o, t in zip(round_order, 't1 t1 t2 t2'.split())]
        [b.new_hand(h) for b, h in zip(bots, (hand11, hand12, hand21, hand22))]
        all_bots.append(bots)
    return all_bots

In [ ]:
#|export
def one_round(bots, round_rank): 
    rm = tr.RoundManager(*bots)
    result = rm.play_round(round_rank)
    return result, rm.history

In [ ]:
faceup = '4s'
round_rank = tr.round_rank(faceup)
bots = get_bots(round_rank, '7s 3c qh'.split(), '5s kh 2s'.split(), '5d 2h ac'.split(), '3d 2c 2d'.split())

In [ ]:
results = parallel(partial(one_round, round_rank=round_rank), bots)

In [ ]:
#|export
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.score = None
        self.team = self.value.split('-')[0] # HACK for now
        
    def __repr__(self):
        return f'Node {self.value} with {len(self.children)} children'

    def dot(self, dot=None):
        if self.score is None: minimax(self)
        dot = dot or graphviz.Digraph(comment="A graph", format="svg")
        dot.node(str(id(self)), f'{self.score} | {self.value}')
        for c in self.children:
            c.dot(dot=dot)
#             dot.edge(str(id(self)), str(id(c)), label=c.value)
            dot.edge(str(id(self)), str(id(c)))
        return dot
    
    def render(self): return self.dot().render('digraph.gv', view=True)  

In [ ]:
#|export
def add_value(root, values):
    v, v_ = values[0], values[1:]
    if v_ is []: return root.children.append(Node(v))
        
    for c in root.children: 
        if c.value == v: add_value(c, v_)
    else: return root.children.append(Node(v))

In [ ]:
#|export
def add_branch(root, values):
    v, v_ = values[0], values[1:]
    # add children if doesn't exist
    for c in root.children:
        if c.value == v: break
    else: 
        c = Node(v)
        root.children.append(c)
        
    if not v_: return c # Last addition
    return add_branch(c, v_)

In [ ]:
#|export
def minimax(root):
    if root.score is not None: return root.score
    scores = [minimax(c) for c in root.children]
    operation = dict(t1=max, t2=min)[root.children[0].team]
    root.score = operation(scores)
    return root.score

In [ ]:
faceup = '4s'
round_rank = tr.round_rank(faceup)
bots = get_bots(round_rank, '7s 3c qh'.split(), '5s kh 2s'.split(), '5d 2h ac'.split(), '3d 2c 2d'.split())

In [ ]:
results = parallel(partial(one_round, round_rank=round_rank), bots)

In [ ]:
#|export
def _round_tree(results):
    root = Node('t1')
    for score, hist in results:
        node = add_branch(root, ['-'.join(o) for o in hist])
        node.score = score
    return root

In [ ]:
#|export
def create_hands(*hands): return [h.split() for h in hands]

In [ ]:
#|export
def round_tree(faceup, *hands):
    bots= get_bots(round_rank, *hands)
    results = parallel(partial(one_round, round_rank=round_rank), bots)
    return _round_tree(results)

In [ ]:
faceup = '4s'
hands = create_hands('7s 3c qh', '5s kh 2s', '5d 2h ac', '3d 2c 2d')
tree = round_tree(faceup, *hands)

In [ ]:
tree.render()

'digraph.gv.svg'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()